# Netengine Example: Fashion MNIST Dataset
Recognizing clothing items in 28x28 grayscale images.

## Imports and Constants

In [7]:
import netengine
from linetimer import CodeTimer

# Size of training and validation datasets.
N_TRAIN = 60_000
N_TEST = 10_000

# Number of neurons in each layer.
LAYOUT = [28 * 28, 400, 150, 10]

# The learning rate for regular weights.
ETA = 0.1

# The learning rate for the bias weights.
ETA_BIAS = 0.02

# The size of one training batch. Between each batch, the accuracy is calculated.
BATCH_SIZE = 10_000

# Define target vectors.
# Each possible classification is represented by one position in the target vector.
TARGETS = []
for i in range(10):
    target = [0] * 10
    target[i] = 1
    TARGETS.append(target)

## Loading Training and Validation Datasets

In [8]:
# Load test data.
with CodeTimer("load test data"):
    with open("../data/fashion_mnist/test_labels", "rb") as file:
        # Skip header.
        file.seek(8)
        test_labels = [TARGETS[int(i)] for i in file.read()][:N_TEST]

    test_images = []
    with open("../data/fashion_mnist/test_images", "rb") as file:
        # Skip header.
        file.seek(16)
        for i in range(N_TEST):
            # Scale to [0, 1].
            test_images.append([pixel / 255.0 for pixel in file.read(28 * 28)])

# Load training data.
with CodeTimer("load training data"):
    with open("../data/fashion_mnist/train_labels", "rb") as file:
        # Skip header.
        file.seek(8)
        train_labels = [TARGETS[int(i)] for i in file.read()][:N_TRAIN]

    train_images = []
    with open("../data/fashion_mnist/train_images", "rb") as file:
        # Skip header.
        file.seek(16)
        for i in range(N_TRAIN):
            # Scale to [0, 1].
            train_images.append([pixel / 255.0 for pixel in file.read(28 * 28)])

Code block 'load test data' took: 578.84202 ms
Code block 'load training data' took: 3039.88780 ms


## Create, Train and Test the Network

Get network instance. Check if CUDA-capable device has been found.

In [9]:
with CodeTimer("create net"):
    net = netengine.Net(LAYOUT, ETA, ETA_BIAS, True)

print()
print(net)
print(f"\n{net.cuda_enabled() = }\n")

Code block 'create net' took: 12.74952 ms

layout: 784 | 400 | 150 | 10
parameters: 375660
eta: 0.1
eta_bias: 0.02

net.cuda_enabled() = True



Train and test the network.

In [ ]:
trained = 0
acc = 0
max_acc = 0
max_acc_at = 0
while True:
    # Train one batch.
    with CodeTimer("train"):
        net.train(train_images, train_labels, BATCH_SIZE, trained % N_TRAIN)

    trained += BATCH_SIZE

    # Test the classification accuracy.
    with CodeTimer("test"):
        acc = net.test(test_images, test_labels)

    # Stop training if the accuracy is decreasing.
    if (1 - acc) > 1.5 * (1 - max_acc):
        print(f"training stopped. accuracy: {100 * acc:.2f}% | trained: {trained}")
        break

    # Save the best accuracy.
    if acc > max_acc:
        max_acc = acc
        max_acc_at = trained

    # Print the results.
    print(f"accuracy: {100 * acc:.2f}% | trained: {trained}")
    print(f"maximum accuracy: {100 * max_acc:.2f}% | at trained: {max_acc_at}\n")

## Results

---

layout: 784 | 400 | 150 | 10

parameters: 375,660

eta: 0.1

eta_bias: 0.02

maximum accuracy: 71.38% | at trained: 120,000